# Cython in Depth

Examples from the O'Reilly Cython (Smith) book

## Interpreted vs compiled execution

To understand how Cython improves the performance of Python code, it is useful to understand the differences in the way that Python's interpreter runs code compared to the way an operating system runs compiled C code.

Before being run, a Python code is automatically compiled to Python _bytecode_. Bytecodes are a set of fundamental codes and instructions that can be understood or executed by the Python virtual machine (VM). As the Python VM abstracts away any platform-specific details, python bytecode can be generated on one platform and run anywhere else with a Python VM.

## Dynamic vs static typing

## Static type declaration with `cdef`

We can use dynamically typed variables in Cython the same way as we can in normal Python.


In [2]:
%load_ext Cython

In [3]:
a = [x + 1 for x in range(12)]
b = a
a[3] = 42.0
assert b[3] == 42.0

a = 13
assert isinstance(b, list)

print(a)
print(b)

13
[1, 2, 3, 42.0, 5, 6, 7, 8, 9, 10, 11, 12]


In Cython, __untyped dynamic variables__ behave exactly like Python variables. The assignment `b = a` allows both `a` and `b` to access the same list object. Modifying the list item with `b[3]` modifies the same list object item referenced by `a`, so th assertion holds true. When `a` is assigned to `13`, `b` still refers to the original list object, and `a` is assigned to a Python integer object. The type of `a` has been changed dynamically, which is perfectly valid Python code.

To **statically type** Python variables, we can use the `cdef` keyword, with a type and a variable name.

In [4]:
%%cython
import cython

cdef int i
cdef int j
cdef float k

Statically typed variables are used just the same way as in C code.

In [5]:
%%cython
j = 0
i = j
k = 12.0
j = 2 * k
assert i != j

Note: Static variables with C types have C semantics which means they follow C-style coercion and casting rules.

In the above example, `i = j` copies the integer data at `j` to the memory location reserved for `i`. The variables `i` and `j` now refer to separate entities, and can evolve separately.

As with C, we can declare several variables of the same type at once:


In [6]:
%%cython
cdef int i, j, k
cdef float price, margin

Inside a function, `cdef` statements are indented and the static variables declared are local to that function. All of these are valid uses of `cdef` to declare local variables in a function `integrate`:

In [7]:
%%cython
def integrate(a, b, f):
    cdef int i
    cdef int N=2000
    cdef float dx, s = 0.0
    dx = (b-a/N)
    for i in range(N):
        s += f(a+i*dx)
    return s * dx

An equivalent way to define multiple C-type variables is inside a `cdef` block:

In [8]:
%%cython
def integrate(a, b, f):
    cdef:
        int i
        int N=2000
        float dx, s=0.0
# ...        

### What about `static` and `const`?

The `static` keyword in C is used to define a variable that extends in lifetime for the entire duration of the program. It is not a valid keyword in Cython. The `const` keyword declares an unmodifiable identifier. We can use `const` in Cython but its use will be explained in a later section. 

## Automatic type inference in Cython

We can use a convenient feature in Cython to automatically deduce statically typed variables in Cython without having to manually use the `cdef` keyword in multiple places. Cython is conservative in that it will not automatically deduce types that do not have definite 1-to-1 mappings with C-types, unless expressly directed to do so. It will however, automatically infer certain types of variables if they have unambiguous mappings to corresponding C-types. For example:


In [9]:
%%cython
def automatic_inference():
    i = 1   # Not converted to a C int (could be float or int, for example)
    d = 2.0 # Automatically treated as a C double
    c = 3 + 4j # Not converted, remains a Python complex
    r = i * d + c # Similarly, remains a python object
    return r

However, we can use the `infer_types` decorator to give Cython more flexibility in infering types in a more intuitive way:

In [10]:
%%cython
def more_inference():
    i = 1   # Not converted to a C int (could be float or int, for example)
    d = 2.0 # Automatically treated as a C double
    c = 3 + 4j # Not converted, remains a Python complex
    r = i * d + c # Similarly, remains a python object
    return r

In this case, `i` is typed as a C `long` type, `c` and `r` are both complex variables. We must take care when using infer_types that integer operations do not overflow and semantics do not change from the untyped version. The `infer_types` directive can be enabled at the function scope or globally.